В этой тетрадке описывается алгоритм решения задачи извлечения из Wiktionary списка идиом немецкого языка, а также, при наличии информации, сопоставления каждой идиоме её части речи, буквального значения, фигурального значения, а также присвоение ссылки на страницу этой идиомы в Wiktionary. 

Для их извлечения информации с Wiktionary будем пользоваться пакетом WiktionaryParser (https://pypi.org/project/wiktionaryparser/).

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import csv
import pandas as pd
from collections import Counter
from tqdm import tqdm

from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

Для начала создаем словарь идиома - ссылка на её страницу в Wiktionary:

In [25]:
url = 'https://en.wiktionary.org/wiki/Category:German_idioms'
dict_of_links = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in soup.select("div[class='mw-category-generated'] a[href^='/wiki/']"):
        if re.search('(?<=>)(.*)(?=<)', str(item)).group(0) == 'Synonymes de langues en français':
            pass
        else:
            collocation = re.search('(?<=>)(.*)(?=<)', str(item)).group(0)
            link = 'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
            dict_of_links[collocation] = link
        #     print('https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0))
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
# dict_of_links

При парсинге попала одна лишняя ссылка, удалим её:

In [26]:
del dict_of_links['German rhetorical questions']

In [27]:
len(dict_of_links)

801

У многих идиом на странице нет их определения, а есть ссылка на альтернативную форму или иное написание. Нам нужно углубляться по ссылке до тех пор, пока мы не попадем на определение. Сделаем функцию для извлечения финального определения:

In [39]:
def final_definition(collocation):
    check = 1
    while check == True:
        collocation_info = parser.fetch(collocation, 'german')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
                for definition in collocation_info[0]['definitions'][0]['text'][1:]:
                    if "alternative form of " in definition.lower():
                        collocation = re.search('(?<=[Aa]lternative form of )(.*)', definition).group(0)
                    elif "alternative spelling of " in definition.lower():
                        collocation = re.search('(?<=[Aa]lternative spelling of )(.*)', definition).group(0)
                    elif "synonym of " in definition.lower():
                        collocation = re.search('(?<=[Ss]ynonym of )(.*)', definition).group(0)
                    else:
                        check = 0
    collocation_info = parser.fetch(collocation, 'german')
    if collocation_info!=[]:
        if collocation_info[0]['definitions']!=[]:
            return collocation_info[0]['definitions'][0]['text'][1:]
        else:
            return ' '

Достаём определения и помещаем в список:  
1. часть речи, соответствующую идиоме  
2. строку, содержащую все определения через точку с запятой  
3. ссылку на страницу идиомы  
4. буквальное значение

Этот список добавляем в список списков.

In [ ]:
from tqdm import tqdm

list_for_file = []
for collocation in tqdm(dict_of_links):
    try:
        collocation_info = parser.fetch(collocation, 'german')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
#                 добавляем идиому:
                list_for_collocation = [collocation]
#     добавляем часть речи, если есть:
                if 'partOfSpeech' in collocation_info[0]['definitions'][0]:
                    list_for_collocation.append(collocation_info[0]['definitions'][0]['partOfSpeech'])
                else:
                    list_for_collocation.append('')
#         добавляем определения, предварительно углубившись до нужной страницы
                list_for_collocation.append("; ".join(final_definition(collocation)))
#     добавляем ссылку на страницу идиомы:
                list_for_collocation.append(dict_of_links[collocation])
#                 добавляем literal meaning, если оно есть
                if collocation_info[0]['etymology']!=[]:
                    if not re.search('(?<=Literally, ")([^"]*)', collocation_info[0]['etymology']):
                        list_for_collocation.append('')
                    else:
                        list_for_collocation.append(re.search('(?<=Literally, ")([^"]*)', \
                                                              collocation_info[0]['etymology']).group(0))
                else:
                    list_for_collocation.append('')
                list_for_file.append(list_for_collocation)
    except AttributeError:
        pass
# list_for_file

In [41]:
len(list_for_file)

769

Получилось достать определения для 769 идиом. Запишем информацию в файл.

In [42]:
with open('German_idioms_1601.tsv', 'w', newline='') as record_file:
    tsv_writer = csv.writer(record_file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(['Collocation', 'Part_of_speech', 'Literal meaning', 'Definition', 'Link'])
    for collocation_info in list_for_file:
        tsv_writer.writerow([collocation_info[0], collocation_info[1], collocation_info[4],\
collocation_info[2], dict_of_links[collocation_info[0]]])

Посмотрим, что получилось.

In [43]:
df = pd.read_csv('German_idioms_1601.tsv', delimiter='\t')
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df
pd.options.display.max_colwidth=100
pd.options.display.max_rows=50
# display(left_align(df[['Collocation','Part_of_speech','Definition']].head(500)))
display(left_align(df))

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,'n Appel und 'n Ei,phrase,an apple and an egg,"(idiomatic, regional) peanuts; a low, minuscule price",https://en.wiktionary.org/wiki/%27n_Appel_und_%27n_Ei
1,ab und zu,adverb,nan,(idiomatic) now and then; occasionally,https://en.wiktionary.org/wiki/ab_und_zu
2,abbügeln,verb,nan,"(transitive, idiomatic) to brush off (To disregard something, to dismiss or ignore someone.)",https://en.wiktionary.org/wiki/abb%C3%BCgeln
3,abfahren,verb,nan,"(intransitive, auxiliary sein) to depart, to leave; (transitive, auxiliary haben) to carry off, to remove; (intransitive, with auf, auxiliary sein, idiomatic, colloquial) to go for; to be crazy about, for; to be a fan of or infatuated with",https://en.wiktionary.org/wiki/abfahren
4,abfärben,verb,nan,"(intransitive, of inks or dyes) to bleed; (intransitive, idiomatic) to rub off [+ auf (jemanden) = on]",https://en.wiktionary.org/wiki/abf%C3%A4rben
5,abgekartete Sache,noun,nan,(idiomatic) put-up affair,https://en.wiktionary.org/wiki/abgekartete_Sache
6,abgekartetes Spiel,noun,nan,"(idiomatic) put-up job, put-up, stitch-up",https://en.wiktionary.org/wiki/abgekartetes_Spiel
7,abkönnen,verb,nan,"(idiomatic, colloquial) to be able to stand or endure something or someone",https://en.wiktionary.org/wiki/abk%C3%B6nnen
8,abwarten und Tee trinken,verb,nan,(idiomatic) to wait and see,https://en.wiktionary.org/wiki/abwarten_und_Tee_trinken
9,Alarm schlagen,verb,nan,"(idiomatic) to sound the alarm, to raise the alarm",https://en.wiktionary.org/wiki/Alarm_schlagen
